In [1]:
import sys
import torch
import tqdm
import numpy as np
import random
import os
import json
sys.path.append('../')
from models.cnn.search_cnn import  SearchCNN, SearchCNNController
from configobj import ConfigObj

In [2]:
basecfg_path = '../configs/mnist/darts.cfg'  #конфиг, на который мы ориентируемся при загрузки модели
cfg = ConfigObj(basecfg_path)
name = cfg['name'] # имя для сохранения результатов
ckp_path = './var_nas/searchs/'+name+'/best_{}.pth.tar' # это шаблон названия сохраненных моделей
seeds = cfg['seeds'].split(';')  # сиды. можно брать из конфига
cfg['device'] = 'cpu'
sc = SearchCNNController(**cfg)

In [9]:
distr = torch.distributions.categorical.Categorical(logits=sc.alpha_reduce[0])    


In [10]:
distr.probs

tensor([[0.1428, 0.1429, 0.1428, 0.1429, 0.1429, 0.1428, 0.1429],
        [0.1428, 0.1427, 0.1428, 0.1428, 0.1429, 0.1431, 0.1430]],
       grad_fn=<SoftmaxBackward>)

In [15]:
distr = torch.distributions.relaxed_categorical.RelaxedOneHotCategorical(logits=sc.alpha_reduce[0], temperature=0.1) 
distr.probs

tensor([[0.1428, 0.1429, 0.1428, 0.1429, 0.1429, 0.1428, 0.1429],
        [0.1428, 0.1427, 0.1428, 0.1428, 0.1429, 0.1431, 0.1430]],
       grad_fn=<SoftmaxBackward>)

In [21]:
def calc_entropy(model):
    entropy = 0
    for alpha in model.alpha_reduce:
        for subalpha in alpha:
            distr = torch.distributions.categorical.Categorical(probs=subalpha)            
            entropy += distr.entropy()
    return entropy

In [28]:
# посмотрим, какова будет энтропия у моделей с низкой температурой и концентрацией плотности у одной из операций
# здесь уже не надо загружать сохраненные параметры. Просто смотрим энтропию, на которую надо ориентироваться
torch.manual_seed(0) # чтобы результат не менялся
for _ in seeds:    
    for alpha in sc.alpha_reduce:
        for subalpha in alpha:
            subalpha.data *= 0
            # подобрал на глаз. Вероятность первой компоненты колеблется от 0.8 до 0.98
            subalpha.data[0] += 0.99 
            subalpha.data[1:] += 0.01/(subalpha.data[1:].shape[0])            
    print (calc_entropy(sc))

tensor(1.0349, grad_fn=<AddBackward0>)
tensor(1.0349, grad_fn=<AddBackward0>)
tensor(1.0349, grad_fn=<AddBackward0>)
tensor(1.0349, grad_fn=<AddBackward0>)
tensor(1.0349, grad_fn=<AddBackward0>)


In [29]:
# посмотрим, какова будет энтропия у моделей с низкой температурой и концентрацией плотности у одной из операций
# здесь уже не надо загружать сохраненные параметры. Просто смотрим энтропию, на которую надо ориентироваться
torch.manual_seed(0) # чтобы результат не менялся
for _ in seeds:    
    for alpha in sc.alpha_reduce:
        for subalpha in alpha:
            subalpha.data *= 0
            # подобрал на глаз. Вероятность первой компоненты колеблется от 0.8 до 0.98   
            subalpha.data += 1/(subalpha.data.shape[0])            
    print (calc_entropy(sc))

tensor(27.2428, grad_fn=<AddBackward0>)
tensor(27.2428, grad_fn=<AddBackward0>)
tensor(27.2428, grad_fn=<AddBackward0>)
tensor(27.2428, grad_fn=<AddBackward0>)
tensor(27.2428, grad_fn=<AddBackward0>)


In [ ]:
# посмотрим, какова будет энтропия у моделей с единичной температурой и концентрацией плотности в центре симплекса
# здесь уже не надо загружать сохраненные параметры. Просто смотрим энтропию, на которую надо ориентироваться
torch.manual_seed(0) # чтобы результат не менялся
for _ in seeds:    
    for alpha in sc.alpha_reduce:
        for subalpha in alpha:
            subalpha.data *= 0
            # подобрал на глаз. Вероятность каждой компоненты колеблется от 0.11 о 0.17
            subalpha.data += torch.randn(subalpha.shape)*0.1
    print (calc_entropy(sc, 0.2))
    # NB: если делать высокую температуру, то энтропия также уменьшается:
    # сэмлпы будут концентрироваться ближе к симплексу

In [ ]:
basecfg_path = '../configs/mnist/darts_test_igr.cfg'  #конфиг, на который мы ориентируемся при загрузки модели
cfg = ConfigObj(basecfg_path)
name = cfg['name'] # имя для сохранения результатов
ckp_path = './var_nas/searchs/'+name+'/best_{}.pth.tar' # это шаблон названия сохраненных моделей
seeds = cfg['seeds'].split(';')  # сиды. можно брать из конфига
cfg['device'] = 'cpu'
sc = SearchCNNController(**cfg)

In [ ]:
#https://arxiv.org/pdf/1912.09588.pdf
def log_det_jac(logits, t):
    k = len(logits)    
    exp_logit = torch.exp(logits/t)
    eps = 0.01
    part1 = -2*(k-1) * torch.log((exp_logit.sum()))
    
    part2 = torch.log(abs(1-t*(exp_logit/(eps+logits)).sum()))
    
    part3 = -(k-1) * torch.log(t)
    
    part4 = (torch.log(abs(logits))+abs(logits/t)).sum() 
    
    return part1 + part2 + part3 +part4
log_det_jac(torch.randn(5), torch.ones(1))

In [ ]:
def calc_entropy(model, temp, entropy_sample_num = 100):
    entropy = 0
    for alpha, cov in zip(model.alpha_reduce, model.alpha_cov_reduce):
        for subalpha, subcov in zip(alpha, cov):
            distr = torch.distributions.lowrank_multivariate_normal.LowRankMultivariateNormal(subalpha,
                                                                                              subcov,
                                                                                              torch.ones(subalpha.shape[0]).to('cpu'))
            subentropy = 0
            for _ in range(entropy_sample_num):
                sample = distr.sample()                
                subentropy=subentropy -distr.log_prob(sample) +log_det_jac(sample, torch.ones(1)*temp)

            subentropy /= entropy_sample_num
            entropy += subentropy
    return entropy

In [ ]:
# посмотрим, какова будет энтропия у моделей с низкой температурой и концентрацией плотности у одной из операций
# здесь уже не надо загружать сохраненные параметры. Просто смотрим энтропию, на которую надо ориентироваться
torch.manual_seed(0) # чтобы результат не менялся
for _ in seeds:    
    for alpha, cov in zip(sc.alpha_reduce, sc.alpha_cov_reduce):
        for subalpha, subcov in zip(alpha, cov):
            subalpha.data *= 0            
            # подобрал на глаз. Вероятность первой компоненты колеблется от 0.8 до 0.98
            subalpha.data += torch.randn(subalpha.shape)
            
            subalpha.data *= 0            
            # подобрал на глаз. Вероятность первой компоненты колеблется от 0.8 до 0.98
            subalpha.data += torch.randn(subalpha.shape)
            
            subcov.data *= 0
            subcov.data += torch.randn(subalpha.shape)*0.1
            subalpha.data[0] += 5 
    print (calc_entropy(sc, 0.2))

In [ ]:
# посмотрим, какова будет энтропия у моделей с единичной температурой и концентрацией плотности в центре симплекса
# здесь уже не надо загружать сохраненные параметры. Просто смотрим энтропию, на которую надо ориентироваться
torch.manual_seed(0) # чтобы результат не менялся
for _ in seeds:    
    for alpha, cov in zip(sc.alpha_reduce, sc.alpha_cov_reduce):
        for subalpha, subcov in zip(alpha, cov):
            subalpha.data *= 0
            # подобрал на глаз. Вероятность каждой компоненты колеблется от 0.11 о 0.17
            subalpha.data += torch.randn(subalpha.shape)*0.1
            
            subcov.data *= 0
            subcov.data += torch.randn(subalpha.shape)*0.1
    print (calc_entropy(sc, 1))
    # NB: если делать высокую температуру, то энтропия также уменьшается:
    # сэмлпы будут концентрироваться ближе к симплексу

In [ ]:
"""
Итог: для сильно фиксированных структур энтропия колеблется в районе -3000,-2500
для максимально энтропийных структур: около -200,-150
среднее значение можно брать около -1000,-700
"""